### Import and Install Libraries

In [13]:
#!pip install sqlalchemy
#!pip install pymysql

In [14]:
import requests
import pandas as pd
from IPython.display import JSON
import my_stuff
from datetime import datetime,timedelta
from pytz import timezone

In [15]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password =my_stuff.my_sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Weather Details

#### API Calls

In [16]:
Response= requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={'Berlin'}&appid={my_stuff.my_API_Key}&units=metric")

In [17]:
Response_= Response.json()


In [18]:
JSON(Response_)

<IPython.core.display.JSON object>

#### Preliminary

In [19]:
#Response_['list'][0]

In [20]:
Response_['list'][0]['main']['temp']

24.49

In [21]:
Response_['list'][0]['main']['humidity']

28

In [22]:
Response_['list'][0]['pop']

0

In [23]:
Response_['list'][0]['weather'][0]['main']

'Clear'

In [24]:
Response_['list'][0]['wind']['speed']

7.58

#### Looping through

In [25]:
city= 'Berlin'
weather_information= {"city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[]}

for info in Response_["list"]:
    weather_information["city"].append(city)
    weather_information["Time_stamp"].append(info.get("dt_txt", None)),
    weather_information["Humidity"].append(info["main"].get("humidity", None))
    weather_information["Temperature"].append(info["main"].get("temp", None))
    weather_information["Outlook"].append(info["weather"][0].get("main", None))
    weather_information["Rain_probability"].append(info.get("rain", {}).get("3h", 0))
    weather_information["Wind_speed"].append(info["wind"].get("speed", None))

weather_information= pd.DataFrame(weather_information)
weather_information.head(2)




,city,Time_stamp,Temperature,Humidity,Outlook,Rain_probability,Wind_speed
0,Berlin,2024-05-17 12:00:00,24.49,28,Clear,0.0,7.58
1,Berlin,2024-05-17 15:00:00,24.42,29,Clear,0.0,7.57


#### Weather info for 24 hours

In [26]:
#Simply slice through the first eight entries

In [27]:
city= 'Berlin'
weather_information= {"city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[]}

for info in Response_["list"][:8]:#Simply slice 
    weather_information["city"].append(city)
    weather_information["Time_stamp"].append(info.get("dt_txt", None)),
    weather_information["Humidity"].append(info["main"].get("humidity", None))
    weather_information["Temperature"].append(info["main"].get("temp", None))
    weather_information["Outlook"].append(info["weather"][0].get("main", None))
    weather_information["Rain_probability"].append(info.get("rain", {}).get("3h", 0))
    weather_information["Wind_speed"].append(info["wind"].get("speed", None))

weather_information= pd.DataFrame(weather_information)



#### Looping for multiple cities

In [28]:
from datetime import datetime,timedelta
from pytz import timezone

berlin_timezone = timezone('Europe/Berlin')
cities=['Berlin','London']

In [29]:
weather_info= {"city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[],
                "Date_retrieved":[]}
for city in cities:
    url= (f"http://api.openweathermap.org/data/2.5/forecast?q={'city'}&appid={my_stuff.my_API_Key}&units=metric")
    response= requests.get(url)
    weather_details = response.json()
    timestamp= datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
    for info in weather_details["list"][:8]:
        weather_info["city"].append(city)
        weather_info["Time_stamp"].append(info.get("dt_txt", None)),
        weather_info["Temperature"].append(info["main"].get("temp", None)),
        weather_info["Humidity"].append(info["main"].get("humidity", None)),
        weather_info["Outlook"].append(info["weather"][0].get("main", None)),
        weather_info["Rain_probability"].append(info.get("rain", {}).get("3h", 0)),
        weather_info["Wind_speed"].append(info["wind"].get("speed", None)),
        weather_info["Date_retrieved"].append(timestamp)

weather_information_2 =pd.DataFrame(weather_info)


In [30]:
city_df=pd.read_sql('city', con=connection_string)
city_df

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,United Kingdom,8799800,51°30′26″N,0°7′39″W


#### Creating a Function

In [31]:
def get_connection_string():
    schema="gans"  
    host="127.0.0.1"       
    user="root"
    password=my_stuff.my_sql_password 
    port=3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

connect = get_connection_string()

In [32]:
def get_weather_information(city_df):
    weather_info= {"city_id":[],
                "city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[],
                "Date_retrieved":[]
                }
    for i,row in city_df.iterrows():#iterrows iterates through each index and row in the dataframe
        url= (f"http://api.openweathermap.org/data/2.5/forecast?q={row['city']}&appid={my_stuff.my_API_Key}&units=metric")
        response= requests.get(url)
        weather_details = response.json()
        timestamp= datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
        for info in weather_details["list"][:8]:
            weather_info["city_id"].append(row["city_id"])
            weather_info["city"].append(row["city"])
            weather_info["Time_stamp"].append(info.get("dt_txt", None)),
            weather_info["Temperature"].append(info["main"].get("temp", None)),
            weather_info["Humidity"].append(info["main"].get("humidity", None)),
            weather_info["Outlook"].append(info["weather"][0].get("main", None)),
            weather_info["Rain_probability"].append(info.get("rain", {}).get("3h", 0)),
            weather_info["Wind_speed"].append(info["wind"].get("speed", None)),
            weather_info["Date_retrieved"].append(timestamp)

    weather_df =pd.DataFrame(weather_info)
    #weather_df.to_sql('weather',if_exists = 'append', con =connect, index = False)
    

In [33]:
#get_weather_information(city_df)

## Flight Details

#### API Calls

In [34]:
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2024-05-08T12:00/2024-05-09T00:00"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"true"}

headers = {
	"X-RapidAPI-Key": my_stuff.my_Rapid_Key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [35]:
response.status_code

200

#### Preliminaries

In [36]:
flight_response= response.json()
JSON(flight_response)


<IPython.core.display.JSON object>

In [37]:
flight_response['arrivals'][0]['arrival']['scheduledTime']['local']

'2024-05-08 11:10+02:00'

In [38]:
flight_response['arrivals'][0]['departure']['airport'].get('name')

'Lisbon'

In [39]:
flight_response['arrivals'][0].get('number')

'FR 1143'

In [40]:
flight_response['arrivals'][0].get('arrival').get('terminal')

'2'

In [41]:
flight_response['arrivals'][0].get('arrival').get('gate')

'Z34'

In [42]:
flight_response['arrivals'][0].get('status')

'Arrived'

#### Looping through

In [43]:
ICAO = 'EDDB'
flight_info = {'flight_number':[],
               'departure_airpot_name': [],
               'arrival_ICAO':[],
               'arrival_terminal':[],
               'arrival_gate':[],
               'arrival_time':[],
               'flight_status':[]
               }

for info in flight_response['arrivals']:
    flight_info['flight_number'].append(info.get('number', None)),
    flight_info['departure_airpot_name'].append(info['departure']['airport'].get('name',None)),
    flight_info['arrival_ICAO'].append(ICAO),
    flight_info['arrival_terminal'].append(info['arrival'].get('terminal', None)),
    flight_info['arrival_gate'].append(info['arrival'].get('gate', None)),
    flight_info['arrival_time'].append(info['arrival']['scheduledTime'].get('local', None)),
    flight_info['flight_status'].append(info.get('status',None))
    
    

flight_info = pd.DataFrame(flight_info)
                                                                          

In [44]:
flight_info['arrival_time']= flight_info['arrival_time'].str[:16] #we can also use [-6] to remove the last six characters

#### Looping for Both Countries

In [45]:
ICAO = ['EDDB', 'EGLL']
berlin_timezone = timezone('Europe/Berlin')
timestamp= datetime.now(berlin_timezone).strftime("%Y-%m-%d")
flight_info = {'flight_number':[],
             'departure_airpot_name': [],
             'arrival_ICAO':[],
             'arrival_terminal':[],
             'arrival_gate':[],
             'arrival_time':[],
             'flight_status':[]
                   }  

for icao in ICAO:
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{timestamp}T00:00/{timestamp}T12:00"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
    headers = {
    	"X-RapidAPI-Key": my_stuff.my_Rapid_Key,
    	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    flight_response= response.json()
    for info in flight_response['arrivals']:
        flight_info['flight_number'].append(info.get('number', None)),
        flight_info['departure_airpot_name'].append(info['departure']['airport'].get('name',None)),
        flight_info['arrival_ICAO'].append(icao),
        flight_info['arrival_terminal'].append(info['arrival'].get('terminal', None)),
        flight_info['arrival_gate'].append(info['arrival'].get('gate', None)),
        flight_info['arrival_time'].append(info['arrival']['scheduledTime'].get('local', None)),
        flight_info['flight_status'].append(info.get('status',None))
        
        
    
flight_info = pd.DataFrame(flight_info)
flight_info['arrival_time']= flight_info['arrival_time'].str[:16]



#### Placing Data in a Function

In [46]:
airport_df = pd.read_sql('airport', con= connection_string)
airport_df

,icao,city_id
0,EDDB,1
1,EGLL,2


In [47]:
def get_flight_info (airport_df):
    
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    tomorrow = tomorrow.strftime('%Y-%m-%d')

    times_list = [{'from':'00:00','to':'11:59'},{'from':'12:00','to':'23:59'}]
    flight_info = {
                 'flight_number':[],
                 'departure_airpot_name': [],
                 'arrival_ICAO':[],
                 'arrival_terminal':[],
                 'arrival_gate':[],
                 'arrival_time':[],
                 'flight_status':[]
                       }  
    for i,row in airport_df.iterrows():
        for time in times_list:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time['from']}/{tomorrow}T{time['to']}"
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
            headers = {
            	"X-RapidAPI-Key": my_stuff.my_Rapid_Key,
            	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.get(url, headers=headers, params=querystring)
            flight_response= response.json()
            for info in flight_response['arrivals']:
                
                flight_info['flight_number'].append(info.get('number', None)),
                flight_info['departure_airpot_name'].append(info['departure']['airport'].get('name',None)),
                flight_info['arrival_ICAO'].append(row['icao']),
                flight_info['arrival_terminal'].append(info['arrival'].get('terminal', None)),
                flight_info['arrival_gate'].append(info['arrival'].get('gate', None)),
                flight_info['arrival_time'].append(info['arrival']['scheduledTime'].get('local', None)),
                flight_info['flight_status'].append(info.get('status',None))
                    
    flight_info = pd.DataFrame(flight_info)
    flight_info['arrival_time']= flight_info['arrival_time'].str[:16]
    #flight_info.to_sql('flights',con=connect, if_exists='append', index=False )

    

In [48]:
#get_flight_info (airport_df)